## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select name from facilities
where membercost <> 0;

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
SELECT COUNT(*) AS facilities_without_member_fee
FROM country_club.facilities
WHERE membercost = 0;

facilities_without_member_fee
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT facid, name, membercost, monthlymaintenance
FROM country_club.facilities
WHERE membercost < 0.20 * monthlymaintenance;

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
SELECT *
FROM country_club.facilities
WHERE facid IN (1, 5);

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT
    name,
    CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS cost_label,
    monthlymaintenance
FROM country_club.facilities;

name,cost_label,monthlymaintenance
Tennis Court 1,expensive,200
Tennis Court 2,expensive,200
Badminton Court,cheap,50
Table Tennis,cheap,10
Massage Room 1,expensive,3000
Massage Room 2,expensive,3000
Squash Court,cheap,80
Snooker Table,cheap,15
Pool Table,cheap,15


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
SELECT firstname, surname
FROM (
    SELECT firstname, surname, ROW_NUMBER() OVER (ORDER BY joindate DESC) AS row_num
    FROM country_club.members
) AS ranked_members
WHERE row_num = 1;


firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
SELECT DISTINCT CONCAT(m.firstname, ' ', m.surname) AS member_name,
                f.name AS court_name
FROM country_club.bookings AS b
JOIN country_club.members AS m ON b.memid = m.memid
JOIN country_club.facilities AS f ON b.facid = f.facid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY member_name;


member_name,court_name
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 2
Burton Tracy,Tennis Court 2
Burton Tracy,Tennis Court 1
Charles Owen,Tennis Court 2
Charles Owen,Tennis Court 1
Darren Smith,Tennis Court 2
David Farrell,Tennis Court 2
David Farrell,Tennis Court 1
David Jones,Tennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
SELECT 
    CONCAT(m.firstname, ' ', m.surname) AS member_name,
    f.name AS facility_name,
    CASE 
        WHEN b.memid = 0 THEN f.guestcost * b.slots 
        ELSE f.membercost * b.slots 
    END AS cost
FROM country_club.bookings AS b
JOIN country_club.facilities AS f ON b.facid = f.facid
LEFT JOIN country_club.members AS m ON b.memid = m.memid
WHERE DATE(b.starttime) = '2012-09-14'
  AND (b.memid = 0 AND f.guestcost * b.slots > 30 OR b.memid != 0 AND f.membercost * b.slots > 30)
ORDER BY cost DESC;



member_name,facility_name,cost
GUEST GUEST,Massage Room 2,320.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Tennis Court 2,150.0
GUEST GUEST,Tennis Court 1,75.0
GUEST GUEST,Tennis Court 1,75.0
GUEST GUEST,Tennis Court 2,75.0
GUEST GUEST,Squash Court,70.0
Jemima Farrell,Massage Room 1,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
SELECT 
    CONCAT(m.firstname, ' ', m.surname) AS member_name,
    f.name AS facility_name,
    CASE 
        WHEN b.memid = 0 THEN f.guestcost * b.slots 
        ELSE f.membercost * b.slots 
    END AS cost
FROM (
    SELECT *
    FROM country_club.bookings
    WHERE DATE(starttime) = '2012-09-14'
) AS b
JOIN country_club.facilities AS f ON b.facid = f.facid
LEFT JOIN country_club.members AS m ON b.memid = m.memid
WHERE (b.memid = 0 AND f.guestcost * b.slots > 30) OR (b.memid != 0 AND f.membercost * b.slots > 30)
ORDER BY cost DESC;


member_name,facility_name,cost
GUEST GUEST,Massage Room 2,320.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Massage Room 1,160.0
GUEST GUEST,Tennis Court 2,150.0
GUEST GUEST,Tennis Court 1,75.0
GUEST GUEST,Tennis Court 1,75.0
GUEST GUEST,Tennis Court 2,75.0
GUEST GUEST,Squash Court,70.0
Jemima Farrell,Massage Room 1,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
SELECT f.name AS facility_name,
       SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END) AS total_revenue
FROM country_club.bookings AS b
JOIN country_club.facilities AS f ON b.facid = f.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;


facility_name,total_revenue
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0
